# Reprezentace slov pomocí vektorů

Už jsme se jednou bavili o tom, jak pracovat s texty a reprezentovat slova pomocí vektorů. Nicméně, vektory, které jsme používali byly velmi dlouhé a navíc nezachycovaly nijak význam slov. Podívejme se ještě na dva jiné způsoby reprezentace slov, které reprezentují podobná slova podobnými vektory a navíc zachycují i další jejich sémantické vlastnosti. Tyto reprezentace slov se potom používají mimojiné v případech, které kombinují rekurentní neuronové sítě se zpracováním textu.

## word2vec

word2vec je celý balík přistupů, jak získat sémanticky zajímavé vektory z textových dat. Jedná se o unsupervised metodu, která funguje na principu podobném autoencoderu. Jsou dvě možnosti, jak vektory naučit: CBOW (continuous bag of words), nebo skip-ngram. Liší se v tom, jestli cílem je předpovědět jedno slovo na základě kontextu (CBOW) nebo kontext na základě jednoho slova (skip-ngram).

### CBOW

Při CBOW přístupu je ve vstupní vrstvě sítě kontext několika slov, každé z nich kódované unárně, jak jsme si ukazovali již dříve (délka vstupního vektoru je tedy $|V||C|$, kde |V| je počet slov a |C| je délka kontextu). Následuje jediná skrytá vrstva, kde je $N$ neuronů (tento počet si můžeme zvolit a určuje délku vektoru, který kóduje každé slovo). Výstupní vrstva obsahuje $|V|$ neuronů a síť se na ní snaží předpovědět slovo, které patří do kontextu kódovaného na vstupní vrstvě. Jako chyba se používá softmax.

### skip-ngram

Skip-Ngram funguje přesně naopak v porovnání s CBOW, jen se prohodí vstupní a vystupní vrstvy, cílem tedy je předpovědět kontext na základě jednoho slova. Ukazuje se, že pro velké datasety tato metoda dává o něco lepší vektory.

Natrénované word2vec vektory se dají stáhnout z https://code.google.com/p/word2vec/ (rozbalený archiv má 3.3 GB).

S natrénovanými vektory si můžeme pohrát a podívat se, co umí.

## GloVe

O podobný výsledek jako word2vec se snaží i GloVe (Global Vectors for Word Representation, http://www-nlp.stanford.edu/pubs/glove.pdf), ale používá expliciní způsob popisu vlastností vektorů. V následujícím předpokádáme, že matice $X_{ij}$ obsahuje počty výskytů slov $i$ a $j$ ve stejném kontextu (vedle sebe), $X_i$ je potom počet všech výskytů slova $i$.

Začíná tím, že chce funkci, která má modelovat vztah mezi slovy $i, j$, když je jako kontext dáno slovo $k$, konkrétně odhaduje podíl pravděpodobností slov $i$ a $j$. Hledá se tedy funkce $$F(w_i, w_j, w_k) = \frac{P_{ik}}{P_{jk}}$$ vektorů, které odpovídají slovům $i, j, k$. GloVe následně přidává další požadavky na tuto funkci, např. to, aby záležela na rozdílu vektorů $w_i - w_j$ a ne na vektorech přímo. Na pravé straně je skalár, a přírozený způsob, jak dostat z vektoru skalár je skalární součin. $F$ by tedy měla být funckí skalárního součinu $(w_i-w_j)^Tw_k$. V dalším kroku chceme, aby nezáleželo na tom, které slovo je v kontextu, a které ne, a tím dojdeme k finální rovnici $$F((w_i-w_j)^Tw_k)=\frac{F(w_i^Tw_k)}{F(w_j^Tw_k)},$$ z toho plyne, že F musí být exponenciela, a tedy, že $w_i^Tw_k = \log(P_{ik})=\log(X_{ik}) - \log(X_i).$

GloVe tedy optimalizuje chybovou funkci $$J = \sum_{i,j=1}^Vf(X_{ij})(w_i^Tw_j+b_i+b_j-\log(X_{ij}))^2,$$ funkce $f$ tu váží vzory tak, aby i málo častá slova měla vliv na výsledek a naopak aby hodně častá slova neovlivňovala výsledek moc.

Autoři GloVe trvdí, že funguje o něco lépe než word2vec. Natrénované vektory je také možné stáhnout (http://nlp.stanford.edu/projects/glove/).

In [52]:
import pandas as pd
from sklearn import neighbors

glove = pd.read_csv("C:/Users/Martin/Downloads/glove.6B.300d.txt", delimiter=' ', skiprows=[8], index_col=0, header=None)

nn = neighbors.NearestNeighbors(algorithm='brute', metric='euclidean')
nn.fit(glove)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='euclidean',
         metric_params=None, n_neighbors=5, p=2, radius=1.0)

In [61]:
glove.iloc[list(nn.kneighbors(glove.loc['blue'])[1][0])].index

Index(['blue', 'red', 'bright', 'purple', 'yellow'], dtype='object', name=0)